In [2]:
import deepcut
from pathlib import Path
from pythainlp import word_tokenize
from matplotlib import pyplot as plt

import re
import pandas as pd
import numpy as np
from numpy import array
from numpy import argmax

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

from nltk import FreqDist, precision, recall, f_measure, NaiveBayesClassifier
from nltk.classify import apply_features
from nltk.classify import util
import collections, itertools
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [3]:
#data = pd.read_csv('train.csv')

In [5]:

texts = []
labels = []
with open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train.txt",encoding="utf8") as f:
    for line in f:
        texts.append(line.strip())
        
with open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train_label.txt",encoding="utf8") as f:
    for line in f:
        labels.append(line.strip())

df = pd.DataFrame({"category":labels, "texts":texts})
df.shape

(24063, 2)

In [6]:
df = df[df.category != 'q']

In [7]:
df.groupby('category').describe()

texts                                                    
          count unique                                     top freq
category                                                           
neg        6140   6129                                 #ERROR!   12
neu       13105  13087                                 #ERROR!   19
pos        4300   4300  แป้งเด็กศรีจันทร์ ซื้อยกโหลได้ที่ไหนคะ    1

In [8]:
df['length'] = df['texts'].apply(word_tokenize).apply(len)
df['length'].describe()

count    23545.000000
mean        27.449310
std         45.216729
min          1.000000
25%          6.000000
50%         12.000000
75%         30.000000
max        758.000000
Name: length, dtype: float64

In [9]:
df

,category,texts,length
0,neu,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ,13
1,neu,คะ,1
2,neg,อิเหี้ยออมทำกูอยากกินเอ็มเค,9
3,neu,😅😅😅,1
4,neu,สวัสดีวันพุธ แนน อะไรนะ,8
...,...,...,...
24058,neg,แม่งควายล้วนนน,5
24059,neg,ดอยสุเทพน้องง ไปหมดแล้วววว #pm25,13
24060,neg,ค่าชุดอาจจะแพงกว่าส่วนลด,6
24061,neu,รัฐต้องการแค่ภาษีครับ,5


In [10]:
df.sample(100)

,category,texts,length
2633,neu,ืทำขายเลยป๋า,5
22082,neu,น่าจะมีบัตรโปรโมชั่นอะไรพิเศษเล็กๆน้อยๆให้กะลู...,17
110,neu,ลอรีอัล รีไวทัลลิฟท์ เดอร์มาลิฟท์เดย์ครีม 50กร...,69
22181,pos,คันต่อไปก็นาวาราเล็งไว้แล้ว,8
22581,neu,เห็นน้องๆ ที่บริษัทใช้กันแล้วบอกว่าดี เลยให้น้...,55
...,...,...,...
4567,neu,อีก 3 วันเปิดจำหน่ายบัตร “G19 Fest เทศกาลดนตรี...,532
4173,neu,ดูดมากๆน้ำท่วมปลอด,5
124,neu,(‘GET 102.5’ ร่วมสร้างปรากฏการณ์เรืองแสงสุดยิ่...,227
1865,neu,มิตซูบิชิ ปาเจโร สปอร์ต ใหม่ ให้ทัน ด้วยระบบ F...,34


In [11]:
from gensim.models import Word2Vec 

In [12]:
y_train = [row for row in df['category']]
Y_train = array(y_train)
Y_train.shape

(23545,)

In [13]:
label_encoder = LabelEncoder()

In [14]:
integer_encoder = label_encoder.fit_transform(Y_train)

In [15]:
integer_encoder

array([1, 1, 0, ..., 0, 1, 1], dtype=int64)

In [40]:
ohe = OneHotEncoder(sparse=False)

In [41]:
onehot_encoded = ohe.fit_transform(integer_encoder.reshape(-1,1))

In [42]:
onehot_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [43]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.texts.values)

In [44]:
word_index = tokenizer.word_index

In [45]:
vocab_size = len(word_index)

In [46]:
vocab_size

85786

In [47]:
df['length'].describe()

count    23545.000000
mean        27.449310
std         45.216729
min          1.000000
25%          6.000000
50%         12.000000
75%         30.000000
max        758.000000
Name: length, dtype: float64

In [48]:
max_length = 758

In [49]:
x_train = tokenizer.texts_to_sequences(df.texts.values)

In [50]:
x_train_padded = pad_sequences(x_train, maxlen=max_length ,padding='post')

In [51]:
x_train_padded.shape

(23545, 758)

In [52]:
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout, Activation
from tensorflow.keras.models import Sequential

In [53]:
model = Sequential()
model.add(Embedding(vocab_size+1,32,input_length=max_length))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 758, 32)           2745184   
_________________________________________________________________
flatten_1 (Flatten)          (None, 24256)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3104896   
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 5,850,467
Trainable params: 5,850,467
Non-trainable params: 0
_________________________________________________________________


In [54]:
from sklearn.model_selection import train_test_split

In [55]:
x_train, x_test, y_train, y_test = train_test_split(x_train_padded, onehot_encoded, test_size=0.3, shuffle=True)

In [56]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((16481, 758), (16481, 3), (7064, 758), (7064, 3))

In [57]:
x_train.shape

(16481, 758)

In [58]:
model.fit(x_train,y_train,
         batch_size=20,
         epochs=3,)

Train on 16481 samples
Epoch 1/3
16481/16481 [==============================] - 35s 2ms/sample - loss: 0.9735 - accuracy: 0.5530
Epoch 2/3
16481/16481 [==============================] - 35s 2ms/sample - loss: 0.5097 - accuracy: 0.8049
Epoch 3/3
16481/16481 [==============================] - 34s 2ms/sample - loss: 0.0666 - accuracy: 0.9826


In [60]:
test_loss,test_acc = model.evaluate(x_test,y_test)
print("\nTest accuracy:",test_acc)

7064/7064 [==============================] - 1s 141us/sample - loss: 1.3449 - accuracy: 0.4843

Test accuracy: 0.48428652


In [66]:
predicions = model.predict(x_test)

TypeError: 'numpy.ndarray' object is not callable